In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from helper_functions import add_technical_indicators, create_dataset

2024-01-16 10:30:21.155859: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-16 10:30:21.155912: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-16 10:30:21.157214: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-16 10:30:21.162931: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-16 10:30:21.769130: W tensorflow/compiler/tf2

In [2]:
#Load the data from the pickle file
with open("data/1y_data.pickle", 'rb') as file:
    data_structure = pickle.load(file)

data_structure = [data_structure[0],
data_structure[1],
data_structure[2],
data_structure[3],
data_structure[4]]

data_array = np.array(data_structure).T

#Assuming the structure is [timestamps, close, high, low, volume]
timestamps = pd.to_datetime(data_array[:, 0], unit='ms')
close_prices = data_array[:, 1]
high_prices = data_array[:, 2]
low_prices = data_array[:, 3]
volumes = data_array[:, 4]

# Combine all features into a DataFrame
data_df = pd.DataFrame({
    'timestamps': timestamps,
    'close': close_prices,
    'high': high_prices,
    'low': low_prices,
    'volume': volumes
})
data_df = add_technical_indicators(data_df)
data_df['hour'] = data_df['timestamps'].dt.hour
data_df['day_of_week'] = data_df['timestamps'].dt.dayofweek

# remove all NaN values
data_df.dropna(inplace=True)
data_df.drop('timestamps', axis=1, inplace=True)
data_df.drop('date', axis=1, inplace=True)
data_df


,close,high,low,volume,SMA_5,SMA_15,EMA_5,EMA_15,RSI,MACD,...,lower_Shadow,high_div_low,trade,shadow1,shadow3,shadow5,mean1,mean2,hour,day_of_week
14,17159.49,17163.26,17151.46,644.57409,17164.716,17147.458667,17161.495947,17151.114156,56.347232,9.195872,...,8.03,1.000688,-18.77,-0.029120,-0.023271,0.012458,-0.005407,322.272485,1,0
15,17155.46,17160.70,17145.52,680.98496,17166.072,17149.197333,17159.483964,17151.657387,53.335936,8.645122,...,9.94,1.000885,-22.80,-0.033481,-0.025786,0.014597,-0.005595,340.475740,1,0
16,17155.85,17172.01,17154.42,593.46650,17160.340,17150.248667,17158.272643,17152.181463,46.881635,8.146214,...,1.43,1.001025,-22.41,-0.037761,-0.010531,0.002410,-0.004061,296.714369,1,0
17,17169.35,17173.08,17155.22,557.72010,17159.552,17150.306667,17161.965095,17154.327530,63.035517,8.739420,...,14.13,1.001041,-8.91,-0.015976,-0.009288,0.025335,0.008024,278.852062,1,0
18,17153.02,17171.08,17148.58,560.13752,17158.634,17152.208000,17158.983397,17154.164089,60.281283,7.801911,...,4.44,1.001312,-25.24,-0.045060,-0.012818,0.007927,-0.002446,280.046230,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105348,46722.84,46791.10,46688.19,109.51639,46817.782,46799.673333,46793.514219,46787.815423,55.148784,30.374832,...,34.65,1.002204,-228.20,-2.083706,-1.460421,0.316391,-0.572015,53.716342,20,1
105349,46768.65,46776.03,46711.11,84.79254,46795.512,46807.175333,46785.226146,46785.419745,51.392490,25.948925,...,57.54,1.001390,-182.39,-2.151015,-2.063979,0.678597,-0.692691,41.320763,20,1
105350,46701.23,46783.96,46671.10,135.79803,46761.540,46803.790667,46757.227431,46774.896027,49.614068,16.807386,...,30.13,1.002418,-249.81,-1.839570,-1.230357,0.221874,-0.504241,66.979230,20,1
105351,46691.39,46719.95,46669.35,84.35015,46735.040,46802.816000,46735.281621,46764.457773,45.133302,8.668717,...,22.04,1.001084,-259.65,-3.078240,-2.739651,0.261292,-1.239180,40.635955,20,1


In [3]:
X1, y1 = create_dataset(data_df)

In [9]:
print(f"Start: {X1[0][0][0]}")
print(f"Finish: {X1[0][99][0]}")

Start: 17159.49
Finish: 17215.58
